In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

#connect to model as normal
llm = OllamaFunctions(
    model="llama3", 
    format="json"
    )

# tell model about functions / tools it can access
llm = llm.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

response = llm.invoke("what is the weather in Singapore?")
print(response)


In [ ]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [ ]:
#create a new tool
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"
print(search.name)
print(search.description)
print(search.args)    

In [ ]:
#create a new tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

print(multiply.name)
print(multiply.description)
print(multiply.args)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


# Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")


# Prompt template
prompt = PromptTemplate.from_template(
    """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde.

Human: {question}
AI: """
)

# Chain
llm = OllamaFunctions(model="llama3", format="json", temperature=0)
structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

In [ ]:
alex = chain.invoke("Describe Alex")
alex

In [ ]:
func_test = chain.invoke("")

In [ ]:
# new attempt
 
from dotenv import load_dotenv
import os

from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Mutliply two integers together."""
    return first_int * second_int

print (multiply.name)
print (multiply.description)
print (multiply.args)
print()
print (multiply)
print()
multiply.invoke({"first_int": 4, "second_int": 5})



load_dotenv()
#print ( os.getenv('OPENAI_API_KEY'))
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
print (llm, "\n")
llm_with_tools = llm.bind_tools([multiply])
print(llm_with_tools)

msg = llm_with_tools.invoke("whats 5 times forty two")
print("msg: \n", msg.tool_calls, "\n\n")

from operator import itemgetter

chain = llm_with_tools 
result = chain.invoke("What's four times 23")
print("This one is just the LLM response - you'll see all the args in JSON format.\n", result, "\n")

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) 
result = chain.invoke("What's four times 23")
print("This one is LLM then an anonymous lamda function that extracts the arguments from the previous response.\n", result, "\n")

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
result = chain.invoke("What's four times 23")
print("And finally - this one takes those args and sends it to the actual multiply function - note this is hardcoded as the logic.\n", result, "\n")


### Let's do same again but in ollama 

In [36]:
# new attempt
 

from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Mutliply two integers together."""
    return first_int * second_int

print (multiply.name)
print (multiply.description)
print (multiply.args)
print()
print (multiply)
print()
multiply.invoke({"first_int": 4, "second_int": 5})


from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

#connect to model as normal
llm = OllamaFunctions(
    model="llama3", 
    format="json",
    keep_alive=-1, # keep the model loaded indefinitely
    )




#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
print (llm, "\n")
llm_with_tools = llm.bind_tools([multiply])
print(llm_with_tools)
#other way
llm_with_tools = llm.bind_tools(
    tools=[
        {
            "name": "multiply",
            "description": "Mutliply two integers together.",
            "parameters": {
                "type": "object",
                "properties": {
                    "first_int": {
                        "type": "int",
                        "description": "The first int you want to multiply" "e.g. x",
                    },
                    "second_int": {
                        "type": "int",
                        "description": "The second_int int you want to multiply" "e.g. y",
                    },
                },
                "required": ["first_int", "second_int"],
            },
        }
    ],
    function_call={"name": "multiply"},
)

msg = llm_with_tools.invoke("What's four times 21")
print (f"msg[{msg}]")
print("msg: \n", msg.tool_calls, "\n\n")

from operator import itemgetter

chain = llm_with_tools 
try:
    result = chain.invoke("What's four times 22")
    print("This one is just the LLM response - you'll see all the args in JSON format.\n", result, "\n")
except ValueError as e:
    print ("failed 22")


chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) 
try:
    result = chain.invoke("What's four times 23")
    print("This one is LLM then an anonymous lamda function that extracts the arguments from the previous response.\n", result, "\n")
except ValueError as e:
    print ("failed 23")

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
try:
    result = chain.invoke("What's four times 24")
    print("And finally - this one takes those args and sends it to the actual multiply function - note this is hardcoded as the logic.\n", result, "\n")
except ValueError as e:
    print ("failed 24")

multiply
multiply(first_int: int, second_int: int) -> int - Mutliply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}

name='multiply' description='multiply(first_int: int, second_int: int) -> int - Mutliply two integers together.' args_schema=<class 'pydantic.v1.main.multiplySchema'> func=<function multiply at 0x79342f20e560>

model='llama3' format='json' keep_alive=-1 

bound=OllamaFunctions(model='llama3', format='json', keep_alive=-1) kwargs={'functions': [StructuredTool(name='multiply', description='multiply(first_int: int, second_int: int) -> int - Mutliply two integers together.', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x79342f20e560>)]}
msg[content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"first_int": 4, "second_int": 21}'}} id='run-314b24e1-8b21-45db-86de-3c25f297c1ca-0']
msg: 
 [] 


failed 22


IndexError: list index out of range

In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

#connect to model as normal
llm = OllamaFunctions(
    model="llama3", 
    format="json"
    )

# tell model about functions / tools it can access
llm = llm.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

response = llm.invoke("what is the weather in Singapore?")

print(response)